In [89]:
!nvidia-smi

Thu Jan  5 12:04:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    32W /  70W |   1216MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [90]:
import torch
import torchvision 
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
import random
import numpy as np

In [91]:
mnist_set = torchvision.datasets.MNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [92]:
class CreateDataSet(Dataset):
  def __init__(self, mnist_data):
    self.data = mnist_data
    self.rand_num = random.randrange(9)
  
  def getrandom(self, num, bas):
    b = np.zeros(bas)
    b[num] = 1
    return b


  def __getitem__(self, index):
    num_image, actual_num = self.data[index]
    random_num = (index+actual_num+self.rand_num)%9
    random_number = self.getrandom(random_num, 10)
    actual_sum = random_num + actual_num
    actual_sum = self.getrandom(actual_sum, 19)
    
    return num_image, random_number, actual_num, actual_sum
  
  def __len__(self):
    return len(self.data)

In [93]:
dataset = CreateDataSet(mnist_set)
train_set, test_set = torch.utils.data.random_split(dataset, [48000,12000])
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size = 64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size=32)

In [ ]:
ctr = 1
for item in train_loader:
  print(f'mnist_num:{item[2]} random_num:{item[1]} actual sum:{item[3]}')
  ctr+=1

  if ctr==2:break


In [94]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1,16,3), nn.ReLU(), nn.BatchNorm2d(16), nn.Dropout(0.1), # 26x26x16
                                   nn.Conv2d(16,32,3), nn.ReLU(), nn.BatchNorm2d(32), nn.Dropout(0.1),# 24x24x32
                                   nn.Conv2d(32,10,1), nn.ReLU(), #24x24x10
                                   nn.MaxPool2d(2, 2), #12x12x16
            
                                   )
        self.conv2 =  nn.Sequential(nn.Conv2d(10,16,3),nn.ReLU(), nn.BatchNorm2d(16), nn.Dropout(0.1), #10x10x16
                                   nn.Conv2d(16,16,3),nn.ReLU(), nn.BatchNorm2d(16), nn.Dropout(0.1),   #8x8x16
                                   nn.Conv2d(16,16,3),nn.ReLU(), nn.BatchNorm2d(16), nn.Dropout(0.1),   #6x6x16
        )

        self.fc1 = nn.Linear(in_features=16 * 6 * 6, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out1 = nn.Linear(in_features=60, out_features=10)
        

        self.fc11 = nn.Linear(in_features=586, out_features=240)
        self.fc21 = nn.Linear(in_features=240, out_features=120)
        self.fc22 = nn.Linear(in_features=120, out_features=60)
        self.fc23 = nn.Linear(in_features=60, out_features=30)
        self.out2 = nn.Linear(in_features=30, out_features=19)

    def forward(self, x, y):
        x = self.conv1(x)
        x = self.conv2(x)

        x = x.reshape(-1, 16*6*6)
        #print(f'shape after flattening:{x.shape}')
        #print(f'shape of random number:{y.shape}')

        x1 = torch.cat([x, y], dim =1)
        #print(f'shape of x1:{x1.shape}')

        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.out1(x)

        x1 = F.relu(self.fc11(x1))
        x1 = F.relu(self.fc21(x1))
        x1 = F.relu(self.fc22(x1))
        x1 = F.relu(self.fc23(x1))
        x1 = self.out2(x1)

        x = F.softmax(x, dim=1)
        x1 = F.softmax(x1, dim=1)

        return x, x1

In [95]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Network().to(device)
#summary(model, input_size=(1, 28, 28))

In [96]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [97]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (num_image, random_number, actual_mnist_num, actual_sum) in enumerate(pbar):
        #random_number = random_number.float()
        num_image, random_number, actual_mnist_num, actual_sum = num_image.to(device), random_number.float().to(device), actual_mnist_num.to(device), actual_sum.float().to(device)
        optimizer.zero_grad()
        output1, output2 = model(num_image, random_number)
        loss1 = F.cross_entropy(output1, actual_mnist_num)
        loss2 = F.cross_entropy(output2, actual_sum)
        
        total_loss = loss = loss1+ (loss2*4)
        total_loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss1={loss1.item():.4f} loss2={loss2.item():.4f} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    loss1 = 0
    loss2 = 0
    test_loss=0
    correct_mnist_data = 0
    correct_sum_predicted = 0

    with torch.no_grad():
        for num_image, random_number, actual_mnist_num, actual_sum in test_loader:
            
            num_image, random_number, actual_mnist_num, actual_sum = num_image.to(device), random_number.float().to(device), actual_mnist_num.to(device), actual_sum.float().to(device)

            output1, output2 = model(num_image, random_number)
            loss1 += F.cross_entropy(output1, actual_mnist_num, reduction='sum').item()  # sum up batch loss
            loss2 += F.cross_entropy(output2, actual_sum, reduction='sum').item()

            test_loss = loss1+loss2

            a = get_num_correct(output1, actual_mnist_num)
            b = get_num_correct(output2, actual_sum.argmax(dim=1))

            correct_mnist_data += a
            correct_sum_predicted += b

    test_loss /= len(test_loader.dataset)

    print(f'Test Set: Total Loss:{test_loss:.3f}\
     MNIST Accuray:{100. * correct_mnist_data / len(test_loader.dataset):.3f}%\t\t \
     Sum Accuracy:{100. * correct_sum_predicted / len(test_loader.dataset):.3f}% \n')
    

In [98]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 50):
    print("epoch %d"% epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch 1


loss1=1.4714 loss2=2.9005 batch_id=749: 100%|██████████| 750/750 [00:15<00:00, 49.54it/s]


Test Set: Total Loss:4.403     MNIST Accuray:96.783%		      Sum Accuracy:11.117% 

epoch 2


loss1=1.4821 loss2=2.8687 batch_id=749: 100%|██████████| 750/750 [00:13<00:00, 53.82it/s]


Test Set: Total Loss:4.386     MNIST Accuray:97.883%		      Sum Accuracy:11.408% 

epoch 3


...

Test Set: Total Loss:3.759     MNIST Accuray:99.100%		      Sum Accuracy:73.633% 

epoch 48


loss1=1.4767 loss2=2.2449 batch_id=749: 100%|██████████| 750/750 [00:14<00:00, 51.94it/s]


Test Set: Total Loss:3.757     MNIST Accuray:99.225%		      Sum Accuracy:73.767% 

epoch 49


loss1=1.4612 loss2=2.2761 batch_id=749: 100%|██████████| 750/750 [00:14<00:00, 52.81it/s]


Test Set: Total Loss:3.760     MNIST Accuray:99.183%		      Sum Accuracy:73.533% 



In [ ]:
model = torch.jit.load(cfg.ckpt_path)
model.eval()